In [1]:
import pandas as pd
from Bio import SeqIO
import gzip
import matplotlib.pyplot as plt
import numpy as np
import math

# Chr01 example

In [2]:
data = pd.read_csv("./data_quintile_round_inNiben261.txt", sep = "\t", header = 0)
Niben261 = pd.read_csv("./Niben261_genes.txt", sep = "\t", header = 0, index_col = 0)
Niben261chr = Niben261[Niben261.index.str.contains("Chr")]
quintile_genes = data.index.tolist() # genes clasificados, con correspondencia en Niben101.
add_genes = list(set(Niben261chr.index.tolist()).difference(quintile_genes))
Niben261add = Niben261chr[Niben261chr.index.isin(add_genes)]
Niben261add["Chr"] = Niben261add.Chr.str.replace("Niben261Chr", "Chr")
Niben261add

C:\Users\milil\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Chr,Start,End
Gene,,,
Niben261Chr01g0001004,Chr01,138648,145484
Niben261Chr01g0003002,Chr01,305466,309127
Niben261Chr01g0004013,Chr01,433168,434139
Niben261Chr01g0005004,Chr01,509493,512123
Niben261Chr01g0005003,Chr01,512164,512450
...,...,...,...
Niben261Chr19g1484002,Chr19,148454367,148463060
Niben261Chr19g1485005,Chr19,148480135,148499120
Niben261Chr19g1485014,Chr19,148530961,148538381


In [3]:
data_complete = data.append(Niben261add)
data_complete

,Chr,Start,End,quintile
Niben261Chr01g0002008,Chr01,266317,268851,1.0
Niben261Chr01g0004001,Chr01,458230,462273,3.0
Niben261Chr01g0004003,Chr01,467574,480423,2.0
Niben261Chr01g0004011,Chr01,465674,466936,1.0
Niben261Chr01g0004012,Chr01,432045,433043,1.0
...,...,...,...,...
Niben261Chr19g1484002,Chr19,148454367,148463060,NaN
Niben261Chr19g1485005,Chr19,148480135,148499120,NaN
Niben261Chr19g1485014,Chr19,148530961,148538381,NaN
Niben261Chr19g1486029,Chr19,148623104,148631519,NaN


In [4]:
# No ejecutar lo de arriba, tarda mucho, se ejecutó una vez y este es el resultado para los cromosomas.
chr_len = {'Niben261Chr01': 186295347,
 'Niben261Chr02': 151625522,
 'Niben261Chr03': 194605305,
 'Niben261Chr04': 189815799,
 'Niben261Chr05': 138451958,
 'Niben261Chr06': 148966868,
 'Niben261Chr07': 152648067,
 'Niben261Chr08': 137820123,
 'Niben261Chr09': 137640743,
 'Niben261Chr10': 138251820,
 'Niben261Chr11': 143669543,
 'Niben261Chr12': 158218613,
 'Niben261Chr13': 153652170,
 'Niben261Chr14': 153778118,
 'Niben261Chr15': 153112746,
 'Niben261Chr16': 148817306,
 'Niben261Chr17': 155116228,
 'Niben261Chr18': 148425545,
 'Niben261Chr19': 148948562}

In [5]:
chr01 = data_complete[data_complete["Chr"] == "Chr01"].sort_values('Start')
chr01

,Chr,Start,End,quintile
Niben261Chr01g0001004,Chr01,138648,145484,NaN
Niben261Chr01g0002008,Chr01,266317,268851,1.0
Niben261Chr01g0003002,Chr01,305466,309127,NaN
Niben261Chr01g0004012,Chr01,432045,433043,1.0
Niben261Chr01g0004013,Chr01,433168,434139,NaN
...,...,...,...,...
Niben261Chr01g1858003,Chr01,185840458,185843056,2.0
Niben261Chr01g1860003,Chr01,186028072,186031716,3.0
Niben261Chr01g1861002,Chr01,186120513,186121051,NaN
Niben261Chr01g1861003,Chr01,186121221,186121439,NaN


In [6]:
def split(length, window):
    intervals = {}
    index = 1
    interval = []
    number_intervals = math.ceil(length/window)
    for i in range(length):
        interval.append(i + 1)
        if index == number_intervals:
            intervals[index] = [intervals[index - 1][1] + 1, length]
            break
        if len(interval) == window:
            intervals[index] = [min(interval), max(interval)]
            index += 1
            interval = []
    return(intervals)

In [48]:
# Intervalos de 100K del Chr01:
window = 1000000 # 1000K.
intervals_chr01 = split(chr_len["Niben261Chr01"], window)

# Ranking de las ventanas por categoría de expresión:
Ejemplo con la categoría 5.

In [54]:
ranking = {}
for interval, values in intervals_chr01.items():
    chr01_values = chr01[(values[0] <= chr01["Start"]) & (chr01["Start"] < values[1])]
    n_genes = len(chr01_values[chr01_values['quintile'].notna()])
    if n_genes != 0:
        list_ranking = []
        for category in [5,4,3,2,1,0]:
            if category == 5 or category == 4 or category == 3:
                list_ranking.append(len(chr01_values[chr01_values["quintile"] == category]))
            else:
                list_ranking.append(-len(chr01_values[chr01_values["quintile"] == category])) 
        ranking[interval] = list_ranking

In [61]:
names = list(ranking.keys())
sorted(names, key=lambda name: ranking[name][1], reverse=True)[:10]

[20, 171, 2, 47, 148, 181, 4, 10, 12, 15]

In [63]:
ranking[171]

[0, 4, 6, -4, -3, 0]

In [32]:
sorted(names, key=lambda name: ranking[name][1], reverse=True)[:10]

IndexError: list index out of range

In [26]:
ranking[1150][0][1]

2

In [44]:
a = {1: [9,8,6,7],
    2: [9,9,6,7],
    3: [10,9,8,4]}

In [45]:
names = list(a.keys())
sorted(names, key=lambda name: a[name][0], reverse=True)[:10]

[3, 1, 2]

In [47]:
[sorted(names, key=lambda name: a[name][ii], reverse=True) for ii in range(4)]

[[3, 1, 2], [2, 3, 1], [3, 1, 2], [1, 2, 3]]